################################################
# KNN
################################################

 1. Exploratory Data Analysis
 2. Data Preprocessing & Feature Engineering
 3. Modeling & Prediction
 4. Model Evaluation
 5. Hyperparameter Optimization
 6. Final Model

In [1]:

import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [ ]:
# 1. Exploratory Data Analysis

In [3]:
df = pd.read_csv("diabetes.csv")
df.head() 

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
df.describe().T 

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [7]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [ ]:
# 2. Data Preprocessing & Feature Engineering

In [10]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

Bu problemin özelinde bir standartlaştırma yapmak gerekir.Kullanacak olduğumuz knn yöntemi uzaklık temelli bir yöntem. Uzaklık temelli yöntemlerde ve gradient descent  temelli yöntemlerde değişkenlerin standart olması elde edilecek sonucların daha hızlı ve daha doğru olmasını sağlayacaktır.

Bu sebeple elimizdeki bağımsız değişkenleri standartlaştırma işlemi yapmamız gerekiyor.

StandardScaler() ı çağırıyoruz ve fit_transform(X) diyerek bağımsız değişkenleri standartlaştırıyoruz. Ve X_scaled diyerek kaydediyorum

In [12]:
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

X_scaled ' ı calıştırdıgımızda bir numpy array dönüyor. Ve değişken değerleri standartlaşmış olsa da elimizdeki numpy array i istediğimiz bilgileri taşımıyor. Yani sutun isimleri yok. Sütun isimlerini eklememiz lazım 

In [13]:
X = pd.DataFrame(X_scaled, columns=X.columns)
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [ ]:
# 3. Modeling & Prediction 

knn modelimizi oluşturacağız.
KNeighborsClassifier derki ; benim aslında bir komşuluk sayısı hyperparametrem var ,ifade etmen lazım. Ama sen onu simdi bilmiyorsun galiba onu pas geç der. Şimdilik pas geçelim.

fit diyorum ve bağımlı değişkenlerim ile bağımsız değişkenlerimi veriyorum.knn modelimizi fit ettik. Modeli kurduk. 

Ne yapmış olduk ? Veri setinde bir öğrenme işlemi gerçekleştirdik. bağımlı ve bağımsız değişken arasındaki ilişkiyi öğrenmiş olduk. 

In [48]:
knn_model = KNeighborsClassifier().fit(X, y)

Şimdi diyelim ki bi tahmin yapmak istiyoruz.İlk önce şöyle yapalım random_user diyerek verisetinden rastgele bi hasta seçelim.

Bir tane örneklem şec diyorum aynı değerleri almak için random_state i 45 diyorum.


Şimdi kurmuş oldugumuz modele bu kullanıcıyı soralım bakalım.Diyabet olup olmadıgını diyoruz.

In [46]:
random_user = X.sample(1, random_state=45)

random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
195,0.342981,1.161295,0.770014,1.283638,1.130518,0.940144,-0.232176,-0.360847


knn_model i derki beni kullanmana izin veriyorum. Ben modeli öğrendim.Ve tahminlerde bulunabiliyorum der.  

Predict metodu derki hangi özellikleri sormamı istersin knn modele. Ben aracıyım.Senin bana verecek oldugun  özelliklere göre gidip knn modele soracağım elimde bu bu bu özellikler var , söyle bakalım bu kişi diyabet hastası mı?

In [16]:
knn_model.predict(random_user)

array([1])

modeli fit etmek yani train etmek ayrı bir süreçtir. Train edilen modeli kullanarak tahminlerde bulunmak ayrı bir süreçtir. 

Predict diyerek model üzerinden tahminde bulunuyoruz.


Bütün veriseti için tahminlerde bulunup, başarımızın ne oldugunu görmek istersek de model basarı kriterlerine gitmek lazım.

In [17]:
# 4. Model Evaluation

Model başarısını değerlendirme işlemini ele alacğız. Tek bir gözlem birimi için tahminde bulunduk. knn modelini kullanarak bütün gözlem birimleri içip tahmin yapıp, bunları bir yerde saklamak lazım


In [49]:
# Confusion matrix için y_pred:
y_pred = knn_model.predict(X)
y_pred

array([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [51]:
# AUC için y_prob: 
y_prob = knn_model.predict_proba(X)[:, 1]
y_prob

# 1 sınıfına ait olma olasılıklarını getiriyoruz.
# Bu olasılık değerleri üzerinden roc eğrisi skoru hesaplayacağız

array([0.8, 0. , 0.8, 0. , 0.4, 0.2, 0.2, 0.4, 0.6, 0.4, 0. , 0.6, 0.4,
       1. , 0.4, 0.4, 0.8, 0.6, 0.2, 0.4, 0.4, 0. , 0.8, 0.6, 0.6, 0.6,
       0.8, 0. , 0.4, 0.4, 0.6, 0.6, 0. , 0. , 0.8, 0.2, 0.4, 0.4, 0.2,
       0.6, 0.8, 0.6, 0.2, 1. , 0.6, 0.8, 0.4, 0. , 0.2, 0.2, 0. , 0. ,
       0. , 1. , 0.4, 0. , 0.8, 0. , 0.6, 0. , 0. , 0.8, 0. , 0.2, 1. ,
       0. , 0.4, 0.8, 0. , 0.6, 0.2, 0.6, 0.6, 0. , 0. , 0. , 0. , 0.2,
       0.8, 0. , 0. , 0. , 0.2, 0. , 0.6, 0.2, 0.2, 0. , 0.6, 0. , 0. ,
       0.4, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0.6, 0.8, 0.2, 0. , 0. ,
       0.2, 0. , 0. , 0.4, 0.2, 0.2, 0.8, 0.6, 0. , 0.2, 0.2, 0.6, 0.6,
       0. , 0. , 0. , 0.8, 0.2, 0. , 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.4,
       0.8, 0.8, 0.8, 0.2, 0.2, 0.2, 0.2, 0. , 0.4, 0.2, 0.4, 0. , 0. ,
       0.6, 0.6, 0. , 0.2, 0.4, 0.2, 0. , 0.2, 0. , 0.6, 0.4, 0.8, 0.8,
       0.2, 0. , 0. , 0.8, 0.2, 0. , 0.2, 0. , 0.4, 0.2, 0. , 0.4, 0.4,
       0. , 0.6, 0.4, 0.2, 0.2, 0. , 0.8, 0.4, 0.8, 0.4, 0.4, 0.

In [52]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



In [23]:
roc_auc_score(y, y_prob)

0.9017686567164179

In [ ]:
Modelin

In [25]:
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
cv_results

{'fit_time': array([0.00532603, 0.00272608, 0.00184584, 0.00123072, 0.00128698]),
 'score_time': array([0.02272701, 0.01509809, 0.00938201, 0.00955224, 0.00820613]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [26]:
cv_results['test_accuracy'].mean()

0.733112638994992

In [27]:
cv_results['test_f1'].mean()

0.5905780011534191

In [28]:
cv_results['test_roc_auc'].mean()

0.7805279524807827

In [29]:
# 1. Örnek boyutu arttıralabilir.
# 2. Veri ön işleme
# 3. Özellik mühendisliği
# 4. İlgili algoritma için optimizasyonlar yapılabilir.

In [30]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [31]:
# 5. Hyperparameter Optimization


In [35]:
knn_model = KNeighborsClassifier()
knn_model.get_params()
knn_params = {"n_neighbors": range(2, 50)}
knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)
knn_gs_best.best_params_


Fitting 5 folds for each of 48 candidates, totalling 240 fits


{'n_neighbors': 17}

In [36]:
# 6. Final Model

In [38]:
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)
knn_final

KNeighborsClassifier(n_neighbors=17)

In [39]:
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [40]:
cv_results['test_accuracy'].mean()


0.7669892199303965

In [41]:
cv_results['test_f1'].mean()


0.6170909049720137

In [42]:
cv_results['test_roc_auc'].mean()


0.8127938504542278

In [45]:
random_user = X.sample(1)
random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
697,-1.141852,-0.685236,-3.572597,-1.288212,-0.692891,-0.887493,-0.661032,-0.956462


In [44]:
knn_final.predict(random_user)


array([0])